In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import re

In [3]:
reviews_train = []
for line in open('../Data/movie_data/full_train.txt', 'r', encoding="utf8"):

    reviews_train.append(line.strip())

reviews_test = []
for line in open('../Data/movie_data/full_test.txt', 'r', encoding="utf8"):

    reviews_test.append(line.strip())

In [12]:
print(len(reviews_train))
print(len(reviews_test))

25000
25000


In [ ]:
reviews_train[0]

In [10]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):

    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]

    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [11]:
reviews_train_clean[0]

'bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my  years in the teaching profession lead me to believe that bromwell highs satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled  at  high a classic line inspector im here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isnt'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)


In [6]:
print(type(X))

<class 'scipy.sparse.csr.csr_matrix'>


In [7]:
print(X[1])

  (0, 292)	1
  (0, 1386)	1
  (0, 1591)	1
  (0, 2102)	1
  (0, 2330)	1
  (0, 2766)	1
  (0, 2861)	1
  (0, 3136)	1
  (0, 3922)	1
  (0, 4356)	1
  (0, 6486)	1
  (0, 6626)	1
  (0, 6892)	1
  (0, 6934)	1
  (0, 7058)	1
  (0, 7556)	1
  (0, 7588)	1
  (0, 7635)	1
  (0, 8332)	1
  (0, 8949)	1
  (0, 8955)	1
  (0, 9575)	1
  (0, 10255)	1
  (0, 10617)	1
  (0, 11014)	1
  :	:
  (0, 85522)	1
  (0, 86727)	1
  (0, 87017)	1
  (0, 87135)	1
  (0, 87137)	1
  (0, 87235)	1
  (0, 87298)	1
  (0, 87901)	1
  (0, 88028)	1
  (0, 88099)	1
  (0, 88106)	1
  (0, 88142)	1
  (0, 88283)	1
  (0, 88524)	1
  (0, 88584)	1
  (0, 88828)	1
  (0, 88859)	1
  (0, 88989)	1
  (0, 89205)	1
  (0, 89268)	1
  (0, 89320)	1
  (0, 89535)	1
  (0, 89971)	1
  (0, 90206)	1
  (0, 90241)	1


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

target = [1 if i < 12500 else 0 for i in range(25000)]

In [16]:
from collections import Counter
print(sorted(Counter(target).items()))

[(0, 12500), (1, 12500)]


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:

    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("F1 for C=%s: %s"
           % (c, accuracy_score(y_val, lr.predict(X_val))))

# F1 for C=0.01: 0.8698131137155527
# F1 for C=0.05: 0.8765727026596593
# F1 for C=0.25: 0.8741214057507987
# F1 for C=0.5: 0.87338336260578
# F1 for C=1: 0.8699408094704847

In [13]:
final_model = LogisticRegression(C=0.05)
final_model.fit(X, target)
print ("Final Accuracy: %s"
       % accuracy_score(target, final_model.predict(X_test)))
print ("Final F1: %s"
       % f1_score(target, final_model.predict(X_test)))

Final Accuracy: 0.88144
Final F1: 0.8818276054541105


In [15]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
# for best_positive in sorted(
#     feature_to_coef.items(),
#     key=lambda x: x[1],
#     reverse=True)[:5]:
#     print (best_positive)

# ('excellent', 0.928354437750993)
# ('perfect', 0.7944277671227645)
# ('great', 0.6745552993465603)
# ('amazing', 0.6164834530323475)
# ('superb', 0.6055919741377176)

for best_negative in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1])[:5]:
    print (best_negative)
#
# ('worst', -1.3679897640418013)
# ('waste', -1.1688808954304515)
# ('awful', -1.0273337471101627)
# ('poorly', -0.8748022410499334)
# ('boring', -0.8591221131175376)

('worst', -1.3679897640418013)
('waste', -1.1688808954304515)
('awful', -1.0273337471101627)
('poorly', -0.8748022410499334)
('boring', -0.8591221131175376)
